In [1]:
import math
# 手册参数（定义详见器件手册）
D_MAGCC = 0.425
I_run = 2e-3
K_AM = 4
K_LC = 25
V_CCR = 330e-3
V_cst_max = 780e-3
V_cst_min = 195e-3
f_sw_min = 1e3
f_sw_max = 100e3
V_vsr = 4.05
I_vsl_run = 225e-6
I_vsl_stop = 80e-6
V_CBC_MAX = 3.2
V_DD_off = 8.1
V_DD_on = 21
V_DSMAX=650

# 指标参数
# 额定输出电压
V_out = 15
# 最小输出电压
V_out_min = 5
# 允许纹波
V_ripple = 75e-3
# 额定输出电流
I_out = 5
# 最小交流输入电压有效值
V_in_min = 90
# 最大交流输入电压有效值
V_in_max = 265
# 额定输出功率
P_out = V_out*I_out
# 变换器的最小开关频率，1.15倍裕量
f_min = f_sw_min*1.15
# 变换器的最大开关频率
f_max = 90e3
# 输入交流频率
f_line = 50
# 预取效率
yita = 0.9


In [2]:
# 自选参数1

# 输出失效电压
V_in_run = 70
# 输入电容放电电压最小值
V_bulk_min = V_in_min*math.sqrt(2)*0.7
# 输入电容放电电压最大值
V_bulk_max = V_in_max*math.sqrt(2)-5
# 估计的谐振周期
t_R = 2e-6
# 输出侧同步整流管压降
V_F = 0.4
# 电缆补偿电压
V_OCBC = 0
# 预取变压器能量转换效率
yita_xfmr = 0.95
# 辅助二极管压降
V_FA = 0.6
# 驱动和开关管延迟
t_D = 50e-9

In [3]:

# 输入功率
P_in = V_out*I_out/yita
# 输入电容
C_bulk = 2*P_in*(0.25+1/2/math.pi*math.asin(V_bulk_min /
                 math.sqrt(2)/V_in_min))/(2*V_in_min**2-V_bulk_min**2)/f_line
# 最大占空比
D_max = 1-(t_R/2*f_max)-D_MAGCC
# 最大原边副边比
N_PS_max = D_max*V_bulk_min/D_MAGCC/(V_out+V_F+V_OCBC)
# 最小辅助绕组副边比
N_AS_min = (V_DD_off+V_FA)/(V_out_min+V_F)


In [4]:
# 自选参数2

# 原边副边比
N_PS = 5.5
# 辅助绕组副边比
N_AS = 2

# 变压器漏感造成的电压上升（有吸收电路，所以近似取为二极管压降多一点，约1V）
V_LK = 1

# 输出瞬态电流
I_tran = I_out*0.5
# 对应的最大电压降落
V_o_delta = V_out*0.2


In [5]:
# CSpin的采样电阻
R_CS = V_CCR*N_PS/2/I_out*math.sqrt(yita_xfmr)
# 初级最大电流
I_pp_max = V_cst_max/R_CS
# 励磁电感
L_P = 2*(V_out+V_F+V_OCBC)*I_out/(yita_xfmr*(I_pp_max**2)*f_max)
N_PA = N_PS/N_AS
# 输出电容
C_out = I_tran*(1/f_sw_min+150e-6)/V_o_delta
# 输出电容的最大等效电阻
R_esr = V_ripple*0.8/I_pp_max/N_PS
# CDDpin要接入的电容
C_dd = (I_run+1e-3)*C_out*V_out/I_out/(V_DD_on-V_DD_off-1)
# VSpin的上节分压电阻
R_s1 = V_in_run*math.sqrt(2)/N_PA/I_vsl_run
# VSpin的下节分压电阻（额定输出时）
R_s2 = R_s1*V_vsr/(N_AS*(V_out+V_F)-V_vsr)
# VSpin的下节分压电阻（5V输出时）
R_s2_max = R_s1*V_vsr/(N_AS*(V_out_min+V_F)-V_vsr)
# 电流采样引脚补偿电阻
R_LC = K_LC*R_s1*R_CS*t_D*N_PA/L_P

# 空载功率估测
# 预取空载效率
yita_sb = 0.6
# stand-by功率
P_sb = P_out*f_min/yita_sb/K_AM**2/f_max
# 预载电阻
R_pl = V_out**2/(P_sb-2.5e-3)

# 缓冲电路计算
# 稳压二极管稳压值
V_DZ = (V_out+V_F)*N_PS
# 吸收电阻最大值
R_sMAX = (V_DSMAX*0.9-V_in_max*math.sqrt(2)-0.7-V_DZ)/I_pp_max


In [8]:
# 一些校核

# 副边开关管电压应力
V_rev = V_in_max*math.sqrt(2)/N_PS+V_out+V_OCBC
D_MAXL = 0.95*(V_out+V_F)*N_PS/((V_out+V_F)*N_PS+V_bulk_min)
D_w = 0.05
# 副边开关管电流峰值
I_peak2 = 2*I_out/(1-D_MAXL-D_w)
# 副边开关管电流有效值
I_rms2=I_peak2*math.sqrt((1-D_MAXL-D_w)/3)
# 辅助二极管电压应力
V_rev2 = V_bulk_max/N_PA+(V_out+V_F)*N_AS
# 开关管电压应力
V_dspk = V_in_max*math.sqrt(2)+(V_out+V_F+V_OCBC)*N_PS+V_LK
# 开关管电流峰值
I_peak1=I_pp_max
# 开关管电流有效值
I_rms1=I_peak1*math.sqrt((1-D_MAXL)/3)
# 开通时间，需大于300ns
t_on_min = L_P/V_in_max/math.sqrt(2)*I_pp_max*V_cst_min/V_cst_max
# 需大于1.2us
t_DMAG_min = t_on_min*V_in_max*math.sqrt(2)/N_PS/(V_out+V_F)
